# Programación estadística con Python - II
----

## Agenda

1. Nuestro pensum de Ciencia de Datos con Python  
<br>
2. Recapitulando el ejercicio anterior  
<br>
3. Comparando variables 

  + 3.1 Distribución acumulada y percentiles
  + 3.2 Valores extremos y diagramas de caja (*boxplots*)
  + 3.3 Diagramas de dispersión (*scatterplots*)
  + 3.4 Covarianza y correlación  
<br>
4. Introducción a la inferencia estadística  

  + 4.1 ¿Qué y por qué? Ejemplo con muestreo repetido (*resampling*)
  + 4.2 Generación de números pseudo-aleatorios
  + 4.3 Variables discretas y continuas
  + 4.4 ¿Qué historias cuentan algunas distribuciones de probabilidad?
  + 4.5 Estimación de parámetros: ¿cómo distinguir un buen modelo según los datos?
  + 4.6 Introducción a la regresión lineal con mínimos cuadrados
  + 4.7 Importancia de EDA antes del ajuste del modelo
  + 4.8 Mis primeros pasos con *Bootstrapping*

## Nuestro *"pensum"* de Ciencia de Datos con Python
[Nuestra Comunidad](https://www.meetup.com/Comunidad-de-Analitica-en-Santo-Domingo-R-Python/)  
<br>
[Votación: Temas de Interés en Ciencia de Datos con Python](https://www.meetup.com/Comunidad-de-Analitica-en-Santo-Domingo-R-Python/polls/1252269/)  
<br>

## Recapitulando el ejercicio anterior

[Notebook en Github](https://nbviewer.jupyter.org/github/pyrdr/charlas/blob/master/intro_eda_python/intro_pypandas.ipynb)

In [2]:
import pandas as pd

df = pd.read_csv('cs_DGII_Nomina_2016.csv', sep=';', encoding = "ISO-8859-1")

df.columns = [x.strip().lower() for x in df.columns.values]

limpiar_salario = lambda x: pd.to_numeric(x.replace(',','').strip(), errors='coerce')
df.salario = df.salario.apply(limpiar_salario)

from unidecode import unidecode
import re

def limpiar_puesto(pstr):
    pstr = unidecode(pstr).strip().lower()
    pstr = re.sub('\(.*?\)','', pstr)
    pstr = pstr.translate({ord('.'): None, ord(','): None})
    return pstr

df['puesto_clean'] = df.puesto.apply(limpiar_puesto)
df.puesto_clean.describe()

count         5416
unique         269
top       conserje
freq           289
Name: puesto_clean, dtype: object

## Introducción a la Inferencia Estadística

* ¿Qué y por qué? Ejemplo con muestreo repetido (*resampling*)

## Nómina de la DGII en el 2016

[Introducción al Análisis Exploratorio de Datos (EDA) con Python](https://github.com/pyrdr/charlas/tree/master/intro_eda_python)

### Pregunta: Según la nómina de la DGII en el 2016, ¿en promedio las mujeres ganan igual que los hombres?

In [4]:
df.groupby(by=['mes','genero']).salario.describe()
ene = df[df.mes == 'ene-16']
feb = df[df.mes == 'feb-16']
feb_por_gen = feb.groupby(by='genero')
total_empleados = feb.salario.count()
feb_por_gen.salario.count().apply(lambda x: x/total_empleados)

feb[feb.salario > 250000][['puesto_clean','salario','genero']]

total_nomina = feb.salario.sum()
feb_por_gen.salario.sum().apply(lambda x: x/total_nomina)

import seaborn as sns
sns.countplot(x='genero',data=feb)

feb_por_gen.salario.sum().apply(lambda x: x/total_nomina).plot(kind='bar')

sns.FacetGrid(data=feb, col='genero', size=8).map(sns.distplot,'salario', kde=False)

sns.factorplot(data=feb, x='genero',y='salario',hue='genero', kind='box',size=6)

import numpy as np
sns.distplot(np.random.normal(feb.salario.mean(),feb.salario.std(),feb.shape[0]))

feb_por_gen.salario.mean()

salario_promedio_f, salario_promedio_m = feb_por_gen.salario.mean()[['F','M']]
salario_promedio_f - salario_promedio_m

salario_promedio_f/salario_promedio_m

(salario_promedio_f/salario_promedio_m - 1)*100

bottom25 = feb[feb.salario <= feb.salario.quantile(.25)]
bottom25_by_gender = bottom25.groupby(by='genero')
bottom25_by_gender.salario.describe()

bottom25_by_gender.salario.mean()['F'] - bottom25_by_gender.salario.mean()['M']

(bottom25_by_gender.salario.mean()['F']/bottom25_by_gender.salario.mean()['M'] - 1) * 100

sns.factorplot(data=bottom25, x='genero',y='salario',hue='genero', kind='box',size=8)

top25 = feb[feb.salario >= feb.salario.quantile(.75)]
top25_by_gender = top25.groupby(by='genero')
top25_by_gender.salario.describe()

top25_by_gender.salario.mean()['F'] - top25_by_gender.salario.mean()['M']

(top25_by_gender.salario.mean()['F']/top25_by_gender.salario.mean()['M'] - 1) * 100

sns.factorplot(data=top25, x='genero',y='salario',hue='genero', kind='box',size=6)

trunc_top25 = feb[(feb.salario >= feb.salario.quantile(.75)) & (feb.salario < feb.salario.max())]
trunc_top25_by_gender = trunc_top25.groupby(by='genero')
trunc_top25_by_gender.salario.describe()

trunc_top25_by_gender.salario.mean()['F'] - trunc_top25_by_gender.salario.mean()['M']

(trunc_top25_by_gender.salario.mean()['F']/trunc_top25_by_gender.salario.mean()['M'] - 1) * 100

sns.factorplot(data=trunc_top25, x='genero',y='salario',hue='genero', kind='box',size=6)

pd.set_option('mode.chained_assignment',None) # incantacion arcana que podemos ignorar por ahora
feb['salario_bin'] = pd.qcut(feb.salario,10,precision=0)
# guardemos esto para graficarlo mas abajo
epg_mean = pd.pivot_table(data=feb,index=['salario_bin'],columns=['genero'],values='salario',aggfunc=np.mean)
epg_mean

from matplotlib import pyplot as plt
# Pandas nos genera el grafico inicial
ax = epg_mean.plot(grid=True,figsize=(10,8))
# Pero con matplotlib modificamos los nombres, las unidades y marcas de los ejes
ax.set_xlabel('Deciles de Salario')
ax.set_ylabel('Salario Promedio')
ticks = ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
# Finalmente agregemos el promedio general como linea punteada para referencia
plt.axhline(feb.salario.mean(),linestyle='dashed')

spg = feb.groupby(by=['puesto_clean']).filter(\
                                              lambda x: \
                                              len(x) > 19 and \
                                                  len(x[x.genero == 'F']) > 0 and \
                                                  len(x[x.genero == 'M']) > 0)
print(spg.shape[0])

spg.groupby(by='genero').salario.describe()

sns.factorplot(data=spg, x='genero',y='salario',hue='genero', kind='box',size=6)

spg_mean_pivot = pd.pivot_table(data=spg,values='salario',index=['puesto_clean'],columns=['genero'],aggfunc=np.mean)
spg_mean_pivot.sort_values('F',inplace=True)
spg_mean_pivot.head(10)

spg_mean_pivot.plot(kind='barh',figsize=(10,15))

spg_diff_pivot = spg_mean_pivot.apply(lambda x: x - x['M'], axis = 1)
spg_diff_pivot.sort_values('F',inplace=True)
spg_diff_pivot['F'].plot(kind='barh',figsize=(10,15))

spg_rel_pivot = spg_mean_pivot.apply(lambda x: (x/x['M'] - 1) * 100, axis=1)
spg_rel_pivot['mas'] = spg_diff_pivot.F.map(lambda x: x > 0)
spg_rel_pivot.sort_values('F',inplace=True)
f = plt.figure(figsize=(12,12))
ax = sns.barplot(data=spg_rel_pivot,y=spg_rel_pivot.index,x='F',hue='mas',orient='h')
ax.set_xlabel('% diferencia promedio')

print("En {:d} de {:d} puestos el salario promedio de las mujeres es mayor que el de los hombres".format(spg_rel_pivot.mas.sum(), spg_rel_pivot.shape[0]))

1451
En 22 de 35 puestos el salario promedio de las mujeres es mayor que el de los hombres



Hasta ahora todo lo que hemos hecho es puramente descriptivo, pero no conclusivo. Pero, ¿podemos inferir algo en términos más rigurosos? ¿Podemos generar una conclusión válida y confiable en términos estadísticos a partir de los datos disponibles?

**_No_** podemos inferir nada concreto en términos generales sobre la "brecha salarial de género" ya que los datos que tenemos son solo de dos meses en _una_ institución. 

**_Si_** podemos inferir sobre los salarios y la "brecha" en esta institución, al menos hasta el momento donde tenemos datos (Ene-Feb 2016).

**_¿Cómo?_**



![Consistencia y Validez](reliability_and_validity.png)